# Building the Clean Modeling Dataset (`df_clean`)

This notebook consolidates the enriched NBA dataset into a single,
leakage-free, model-ready table.

All feature cleaning, target construction, and temporal alignment
are performed here. No modeling assumptions are made at this stage.


In [7]:
from pathlib import Path
import pandas as pd
# ------------------------------------------------------------------
# Project root (robust, no pyproject.toml needed)
# Looks for common "project markers": .git, data/, notebooks/, README.md
# ------------------------------------------------------------------

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR

MARKERS = [
    ".git",
    "data",
    "notebooks",
    "README.md",
]

def is_project_root(p: Path) -> bool:
    return any((p / m).exists() for m in MARKERS)

while not is_project_root(PROJECT_ROOT):
    if PROJECT_ROOT.parent == PROJECT_ROOT:
        raise RuntimeError(
            "Project root not found. Run this notebook from inside the repo "
            "(a folder containing one of: .git, data/, notebooks/, README.md)."
        )
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
DATA_PROCESSED_FINAL = DATA_PROCESSED / "players" / "final"
DATA_INTERIM = PROJECT_ROOT / "data" / "interim"
DATA_INTERIM.mkdir(parents=True, exist_ok=True)

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_PROCESSED:", DATA_PROCESSED)


PROJECT_ROOT: c:\Users\Luc\Documents\projets-data\nba-awards-predictor
DATA_PROCESSED: c:\Users\Luc\Documents\projets-data\nba-awards-predictor\data\processed


In [8]:
ENRICHED_PATH = DATA_PROCESSED_FINAL / "all_years_enriched.parquet"
assert ENRICHED_PATH.exists(), f"Missing file: {ENRICHED_PATH}"

df = pd.read_parquet(ENRICHED_PATH)
print("Raw shape:", df.shape)
display(df.head(3))


Raw shape: (13843, 436)


,Rk,Player,Age,Team,Pos,G,GS,MP,FG,FGA,...,raptor__o_raptor_rank,raptor__player_id,raptor__player_name,raptor__poss,raptor__raptor_defense,raptor__raptor_offense,raptor__raptor_rank,raptor__raptor_total,raptor__season,raptor__war_total
0,199.0,A 1/2an Tabak,25.0,TOR,C,67.0,18.0,19.9,3.4,6.2,...,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,139.0,A arA<<nas MarAiulionis,31.0,SAC,SG,53.0,0.0,19.6,3.3,7.3,...,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,202.0,A.C. Green,32.0,PHO,SF,82.0,36.0,25.8,2.6,5.4,...,155.0,greenac01,A.C. Green,4316.0,-0.6,-0.6,174.0,-1.3,1996.0,1.648381


## 1) Basic hygiene

- Ensure `season` exists and has plausible range.
- Ensure the dataset granularity is **one row per (player_key, season)**.
- Drop known rare duplicates (keep the most complete row).


In [9]:
# -----------------------------
# Basic structural checks
# -----------------------------
assert "season" in df.columns, "Missing `season` column."
print("Seasons:", int(df["season"].min()), "→", int(df["season"].max()), "| n =", df["season"].nunique())

PLAYER_KEY = "player_key" if "player_key" in df.columns else "Player"
assert PLAYER_KEY in df.columns, f"Missing player identifier ({PLAYER_KEY})."

dup_mask = df.duplicated(subset=[PLAYER_KEY, "season"], keep=False)
n_dups = int(dup_mask.sum())
print("Duplicate player-season rows:", n_dups)

if n_dups:
    # keep row with max non-null count
    df_dups = df.loc[dup_mask].copy()
    df_dups["_nn"] = df_dups.notna().sum(axis=1)
    df_dups = df_dups.sort_values([PLAYER_KEY, "season", "_nn"], ascending=[True, True, False])

    df_keep = df_dups.drop_duplicates(subset=[PLAYER_KEY, "season"], keep="first").drop(columns=["_nn"])
    df_rest = df.loc[~dup_mask]
    df = pd.concat([df_rest, df_keep], ignore_index=True)

    # re-check
    assert df.duplicated(subset=[PLAYER_KEY, "season"]).sum() == 0, "Still have duplicates after cleanup."


Seasons: 1996 → 2024 | n = 29
Duplicate player-season rows: 2


## 2) Minimal column cleanup

We keep `df_clean` as the **analysis-ready table** (identifiers, a few categorical fields, plus numeric stats).  
We remove:
- obvious duplicates (e.g., duplicated age columns),
- redundant year fields (keep `season`),
- raw award strings / helper columns that are not used.

> The **model-ready** matrices are built later from `df_clean` (in notebook 03).


In [10]:
# -----------------------------
# Column cleanup (project-specific)
# -----------------------------
DROP_FROM_DF_CLEAN = [
    # redundant year field
    "year",

    # duplicate age/team fields from nba_api (kept: Age, Team)
    "AGE", "TEAM_ABBREVIATION",

    # award strings from different stat tables (not useful; keep binary winner labels separately)
    "tot_Awards", "p36_Awards", "p100_Awards", "adv_Awards", "adj_Awards", "shot_Awards",
]

before = df.shape[1]
df_clean = df.drop(columns=[c for c in DROP_FROM_DF_CLEAN if c in df.columns]).copy()
print(f"Dropped {before - df_clean.shape[1]} columns from df_clean")
print("df_clean shape:", df_clean.shape)

# sanity
assert "season" in df_clean.columns
assert "Age" in df_clean.columns
assert "Team" in df_clean.columns
assert "year" not in df_clean.columns


Dropped 9 columns from df_clean
df_clean shape: (13842, 427)


## 3) Leakage columns (kept in df_clean, excluded from X)

We keep award outcomes in `df_clean` for auditing and target construction, but we **never** include them in the feature matrix `X_df`.

This makes the pipeline transparent:
- `df_clean` contains *everything we know*
- `X_df_*` contains *only allowable inputs*


In [11]:
# -----------------------------
# Quick leakage inventory (for visibility only)
# -----------------------------
import re

LEAK_PATTERNS = [
    r".*_rank$",
    r"^is_.*",                      # includes is_*_winner
    r"^all_nba_.*", r"^all_def_.*", r"^all_rookie_.*",
    r"^has_.*consideration$",
    r"^pct_is_.*winner$",
    r"^pct_is_.*_winner$",
]

def is_leak_col(c: str) -> bool:
    return any(re.match(p, c) for p in LEAK_PATTERNS)

leak_cols = [c for c in df_clean.columns if is_leak_col(c)]
print("Leakage-like columns found:", len(leak_cols))
print(leak_cols[:30])


Leakage-like columns found: 38
['is_rookie', 'is_mvp', 'mvp_rank', 'is_dpoy', 'dpoy_rank', 'is_roy', 'roy_rank', 'is_mip', 'mip_rank', 'is_smoy', 'smoy_rank', 'is_cpoy', 'cpoy_rank', 'is_mvp_winner', 'is_dpoy_winner', 'is_roy_winner', 'is_mip_winner', 'is_smoy_winner', 'is_cpoy_winner', 'is_all_star', 'is_all_nba', 'all_nba_team', 'is_all_def', 'all_def_team', 'is_all_rookie', 'all_rookie_team', 'pct_is_mvp_winner', 'pct_is_dpoy_winner', 'pct_is_roy_winner', 'pct_is_mip_winner']


## 4) Save outputs


In [12]:
# -----------------------------
# Save df_clean + a quick column list for manual review
# -----------------------------

DF_CLEAN_PATH = DATA_INTERIM / "df_clean.parquet"
COLS_PATH = DATA_INTERIM / "df_clean_columns.csv"

df_clean.to_parquet(DF_CLEAN_PATH, index=False)
pd.Series(df_clean.columns, name="column").to_csv(COLS_PATH, index=False)

print("Saved:", DF_CLEAN_PATH)
print("Saved:", COLS_PATH)


Saved: c:\Users\Luc\Documents\projets-data\nba-awards-predictor\data\interim\df_clean.parquet
Saved: c:\Users\Luc\Documents\projets-data\nba-awards-predictor\data\interim\df_clean_columns.csv
